In [1]:
import cv2
import matplotlib.pyplot as plt

import math

In [2]:
def show_img(opencv_image):
    """
    Converts Opencv Image via Matplotlib
    
    """
    
    plt_image = opencv_image
    if opencv_image.shape[-1] == 3:
        plt_image = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
    
        
    plt.imshow(plt_image)
    plt.show()

In [3]:
def get_scaled_dims(height, width):
    """
    Calculate the minimum down-scaling required to make dimensions multiple of 32
    """
    new_height = ((height//32)) * 32
    new_width = ((width//32)) * 32
    
    return new_height, new_width

In [4]:
def decode(scores, geometry, scoreThresh):
    detections = []
    confidences = []

    ############ CHECK DIMENSIONS AND SHAPES OF geometry AND scores ############
    assert len(scores.shape) == 4, "Incorrect dimensions of scores"
    assert len(geometry.shape) == 4, "Incorrect dimensions of geometry"
    assert scores.shape[0] == 1, "Invalid dimensions of scores"
    assert geometry.shape[0] == 1, "Invalid dimensions of geometry"
    assert scores.shape[1] == 1, "Invalid dimensions of scores"
    assert geometry.shape[1] == 5, "Invalid dimensions of geometry"
    assert scores.shape[2] == geometry.shape[2], "Invalid dimensions of scores and geometry"
    assert scores.shape[3] == geometry.shape[3], "Invalid dimensions of scores and geometry"
    height = scores.shape[2]
    width = scores.shape[3]
    for y in range(0, height):

        # Extract data from scores
        scoresData = scores[0][0][y]
        x0_data = geometry[0][0][y]
        x1_data = geometry[0][1][y]
        x2_data = geometry[0][2][y]
        x3_data = geometry[0][3][y]
        anglesData = geometry[0][4][y]
        for x in range(0, width):
            score = scoresData[x]

            # If score is lower than threshold score, move to next x
            if(score < scoreThresh):
                continue

            # Calculate offset
            offsetX = x * 4.0
            offsetY = y * 4.0
            angle = anglesData[x]

            # Calculate cos and sin of angle
            cosA = math.cos(angle)
            sinA = math.sin(angle)
            h = x0_data[x] + x2_data[x]
            w = x1_data[x] + x3_data[x]

            # Calculate offset
            offset = ([offsetX + cosA * x1_data[x] + sinA * x2_data[x], offsetY - sinA * x1_data[x] + cosA * x2_data[x]])

            # Find points for rectangle
            p1 = (-sinA * h + offset[0], -cosA * h + offset[1])
            p3 = (-cosA * w + offset[0],  sinA * w + offset[1])
            center = (0.5*(p1[0]+p3[0]), 0.5*(p1[1]+p3[1]))
            detections.append((center, (w,h), -1*angle * 180.0 / math.pi))
            confidences.append(float(score))

    # Return detections and confidences
    return [detections, confidences]

### Parameters

In [5]:
#Model path
model = 'frozen_east_text_detection.pb'

#Model Parameters
confThreshold = 0.5
nmsThreshold = 0.4

#Input Parameters
img_path = 'images/sign.jpg'
inpWidth = None
inpHeight = None

### Load model

In [6]:
# Load network
net = cv2.dnn.readNet(model)

outNames = []
outNames.append("feature_fusion/Conv_7/Sigmoid")
outNames.append("feature_fusion/concat_3")

### Prepare Input

In [7]:
#Read the image
image = cv2.imread(img_path)

# Get image height and width
height_ = image.shape[0]
width_ = image.shape[1]

#Automatically calculate minimal scaling required..
if inpHeight is None and inpWidth is None: 
    inpHeight, inpWidth = get_scaled_dims(height_, width_)
    image = cv2.resize(image, (inpWidth,inpHeight), interpolation=cv2.INTER_AREA)
    rW,rH = 1,1
    
else:
    rW = width_ / float(inpWidth)
    rH = height_ / float(inpHeight)

# Create a 4D blob from image.
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, mean=(123.68, 116.78, 103.94), swapRB=True, crop=False)

### Run the model

In [8]:
net.setInput(blob)
outs = net.forward(outNames)
t, _ = net.getPerfProfile()

### Get scores and geometry

In [9]:
scores = outs[0]
geometry = outs[1]
[boxes, confidences] = decode(scores, geometry, confThreshold)

### Post-processing to remove weak & overlapping bounding-boxes

In [10]:
# Apply Non-Max Suppression (NMS)
indices = cv2.dnn.NMSBoxesRotated(boxes, confidences, confThreshold,nmsThreshold)

# #Ignore Non-Max Suppression
# indices = [[i] for i in range(len(boxes))]

for i in indices:
    # get 4 corners of the rotated rect
    vertices = cv2.boxPoints(boxes[i[0]])
    # scale the bounding box coordinates based on the respective ratios
    for j in range(4):
        vertices[j][0] *= rW
        vertices[j][1] *= rH
    
    #Draw the bounding boxes
    for j in range(4):
        p1 = (vertices[j][0], vertices[j][1])
        p2 = (vertices[(j + 1) % 4][0], vertices[(j + 1) % 4][1])
        cv2.line(image, p1, p2, (0, 255, 0), 1)

In [11]:
cv2.imwrite('text_detection_result.jpg', image)

True